In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os, sys
import time,requests
from bs4 import BeautifulSoup
import keyboard

In [5]:
filename = 'test.mp3'
delayTime = 2
audioToTextDelay = 5
# byPassUrl = 'https://www.google.com/recaptcha/api2/demo'
byPassUrl = 'https://goload.io/download?id=MTE0MjAz&typesub=Gogoanime-DUB&title=Naruto+%28Dub%29+Episode+50'
googleIBMLink = 'https://speech-to-text-demo.ng.bluemix.net/'
option = webdriver.ChromeOptions()
option.add_argument('--disable-notifications')
option.add_argument("--mute-audio")

In [6]:
def saveFile(content,filename):
    with open(filename, "wb") as handle:
        for data in content.iter_content():
            handle.write(data)
            
def speech_text(path):
    driver.execute_script('''window.open("","_blank");''')
    driver.switch_to.window(driver.window_handles[1])
    print('1')
    driver.get(googleIBMLink) # open the translator link
    time.sleep(delayTime)
    print('2')
    root = driver.find_element_by_id('root').find_elements_by_class_name('dropzone _container _container_large')
    btn = driver.find_element(By.XPATH, '//*[@id="root"]/div/input')
    time.sleep(delayTime)
    btn.send_keys(f'C:\\Users\\{os.getlogin()}\\Downloads' + '/' + filename) # upload the files
    
    time.sleep(delayTime)
    time.sleep(audioToTextDelay)
    print('3')
    text = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[7]/div/div/div').find_elements(By.TAG_NAME, 'span') # save the text data
    time.sleep(delayTime)
    result = " ".join( [ each.text for each in text ] )
    print("4")
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print("5")
    os.remove(f'C:\\Users\\{os.getlogin()}\\Downloads' + '/' + filename)
    return result

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
driver.get(byPassUrl)
time.sleep(1)
outeriframe = driver.find_element(By.TAG_NAME, 'iframe')
time.sleep(1)
# print('Going to click')
outeriframe.click()
# time.sleep(1)
allIframesLen = driver.find_elements(By.TAG_NAME, 'iframe')
# print('WHAT is this')
time.sleep(1)
audioBtnFound = False
audioBtnIndex = -1

for index in range(len(allIframesLen)):
    driver.switch_to.default_content()
#     print('Checking this')
    iframe = driver.find_elements(By.TAG_NAME, 'iframe')[index]
    driver.switch_to.frame(iframe)
    driver.implicitly_wait(delayTime)
    try:
        audioBtn = driver.find_element(By.ID, 'recaptcha-audio-button') or driver.find_element(By.ID, 'recaptcha-anchor')
        audioBtn.click()
#         print('WANT TO CLICK')
        audioBtnFound = True
        audioBtnIndex = index
        break
    except Exception as e:
        pass
    
if audioBtnFound == True:
    try:
        while True:
            href = driver.find_element(By.ID, 'audio-source').get_attribute('src')
            response = requests.get(href, stream=True)
            saveFile(response,filename)
            text = speech_text(f'C:\\Users\\{os.getlogin()}\\Downloads' + '/' + filename)
            print(text)
            driver.switch_to.default_content()
            iframe = driver.find_elements(By.TAG_NAME, 'iframe')[audioBtnIndex]
            driver.switch_to.frame(iframe)
            inputbtn = driver.find_element(By.ID, 'audio-response')
            inputbtn.send_keys(response)
            inputbtn.send_keys(Keys.ENTER)
            time.sleep(2)
            errorMsg = driver.find_elements(By.CLASS_NAME, 'rc-audiochallenge-error-message')[0]
            if errorMsg.text == "" or errorMsg.value_of_css_property('display') == 'none':
                print("Success")
                break
    except Exception as e:
        pass
#         print('!!!!!!!!Try after some time!!!!!!!!')
else:
    pass
#     print('!!!!!!!!There is an error while proceeding. Try again!!!!!!!!.')
    

C:\Users\Hemant\AppData\Local\Temp\ipykernel_5748\848637173.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)
